## Pandas (Combining Datasets - Merge and Join)

El principal interface para realizar estas operativas es pd.merge()

### Relational Algebra

El comportamiento de pd.merge() es un subconjunto de lo que se conoce como álgebra relacional, que es una serie de reglas para manipular datos relacionados y forma la base de las operaciones disponibles en la 
la mayoría de las bases de datos.

La fortaleza del algebra relacional que es propone una serie de operaciones sencillas a partir de las cuales se pueden construir otras operaciones más complejas.

Pandas implementa estas a través de pd.merge() y el método join() de las Series y DataFrames.

### Categorías de Joins

La función pd.merge() implementa 3 tipos de joins:

* one-to-one
* many-to-one
* many-to-many

Estos tres tipos de joins, sin utilizar argumentos adicionales, pueden ser utilizados en DataFrame y otras herramientas de Panda, para implementar un rango amplio de funcionalidades.

En la práctica, los datasets no están tan limpios como para trabajar así con ellos y dejar que Pandas detecte la columna clave por la cuál hacer la unión no es buena idea. Para ello, existen una serie de opciones que permite afinar las operaciones con joins.


In [1]:
import numpy as np
import pandas as pd 

In [2]:
pd.__version__ #versión de pandas

'0.23.0'

#### One-to-one joins

Se trata de un join parecido al que se hace con la función pd.concat().

In [3]:
df1 = pd.DataFrame({'employee':['Bob', 'Jake', 'Lisa', 'Sue'],
                   'group':['Accounting', 'Engineering', 'Engineering', 'HR']})

In [5]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [9]:
df2 = pd.DataFrame({'employee':['Lisa', 'Bob', 'Jake', 'Sue'],
                   'hire_date':[2004, 2008, 2012, 2014]})

In [10]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un simple dataframe, usamos la 
función pd.merge()

In [12]:
df3 = pd.merge(df1, df2)

La función pd.merge() reconoce que en cada uno de los dataframe hay una
columna llamada "employee" y automáticamente realiza el join utilizando esta columna. 

Este tipo de join, en general, descarta el índice. A tenerlo en cuenta si es necesario; para ello debería utilizarse "merges by index"

#### Many-to-one joins

Se trata de un join en el cuál una de las columnas clave tiene valores duplicados.  En este caso, el resultado será un nuevo Dataframe, que preservará las entradas duplicadas.

In [13]:
df4 = pd.DataFrame({'group':['Accounting', 'Engineering', 'HR'],
                   'supervisor':['Carly', 'Guido', 'Steve']})

In [15]:
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [16]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [14]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


En este caso la columna para realizar el merge es group, y hay valores duplicados. Se repite el valor del supervisor en aquellas filas donde coincide el grupo.

#### Many-to-many joins

Se trata de un join en el cuál la columna clave presenta duplicados en ambos dataframes.  En este caso, el resultado será un nuevo Dataframe, que preservará las entradas duplicadas en un merge many to many.

In [17]:
df5 = pd.DataFrame({'group':['Accounting', 'Accounting', 'Engineering','Engineering', 'HR', 'HR'],
                   'skills':['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']})

In [18]:
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [19]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [20]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


En este caso, la columna clave es group, y por cada uno de los empleados, genera una fila con cada uno de los skills.

#### Specification of the Merge Key

Ya hemos visto el comportamiento por defecto de la función pd.merge(), que busca las columnas de matching a partir del nombre de estas. A menudo, las columnas no machean tan fácilmente y tenemos que utilizar una serie de opciones que ofrece la función.

##### The on keyword

Lo más sencillo, es definir explicitamente el nombre de la columna con la que hacer el join, a través del argumento "on:", que acepta un nombre de columna o una lista de nombres de columnas.


In [21]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [22]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [28]:
# especificamos el nombre de la columna. En este caso debe ser
# igual en ambos dataframes
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [29]:
# si no encuentra ambas columnas en los dos df, se generará un error
pd.merge(df1, df2, on='group')

KeyError: 'group'

##### The left_on and right_on keywords

En algunos casos, al combinar dos dataframes, el nombre de la columna clave no coincide entre ambos. Podemos utilizar estas keywords, para determinar explicitamente el nombre de las dos columnas.

In [30]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [31]:
df3 = pd.DataFrame({'name':['Lisa', 'Bob', 'Jake', 'Sue'],
                   'salary':[70000, 80000, 120000, 90000]})

In [33]:
# determinamos el nombre de las columnas para hacer el join
pd.merge(df1, df3, left_on='employee', right_on='name')

,employee,group,name,salary
0,Bob,Accounting,Bob,80000
1,Jake,Engineering,Jake,120000
2,Lisa,Engineering,Lisa,70000
3,Sue,HR,Sue,90000


En el resultado del nuevo DataFrame, tendremos dos columnas repetidas,
una por "employee" y otra por "name", mostrando información redundante.

Podemos eliminarla usando el método drop(), especificando el nombre
de la columna a eliminar y el eje en el que está

In [36]:
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,80000
1,Jake,Engineering,120000
2,Lisa,Engineering,70000
3,Sue,HR,90000


##### The left_index and right_index keywords

En algunos casos, más que querer hacer el join por una columna en concreto, queremos hacerlo a través de los índices de los df. Para 
estos casos usamos estas dos keywords

In [38]:
# creamos un df cambiando el índice por la columna "employee"
df1a = df1.set_index('employee')

In [40]:
# la columna pasa a ser el índice y ya no se muestra de forma
# redundante
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [41]:
# creamos un df cambiando el índice por la columna "employee"
df2a = df2.set_index('employee')

In [43]:
# la columna pasa a ser el índice y ya no se muestra de forma
# redundante
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [46]:
# hacemos el join por los índices
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


DataFrames implementa el método join(), el cuál ejecuta un merge 
basado en índices, haciendo lo mismo que el caso anterior

In [48]:
# hacemos un join basado en índices
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


También podemos mezclar joins entre índices y columnas

In [49]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [50]:
df3

,name,salary
0,Lisa,70000
1,Bob,80000
2,Jake,120000
3,Sue,90000


In [51]:
# merge mezclando indices y columnas
pd.merge(df1a, df3, left_index=True, right_on="name")

,group,name,salary
1,Accounting,Bob,80000
2,Engineering,Jake,120000
0,Engineering,Lisa,70000
3,HR,Sue,90000


En este caso, la columna name se mantiene

###  Specifying Set Arithmetic for Joins

En todos los ejemplos anteriores, hemos pasado por alto una importante consideración al ejecutar los joins. El tipo de aritmética utilizada en ellos. En los ejemplos anteriores, todos los valores que aparecían en una de los df aparecían en el otro.

In [52]:
df6 = pd.DataFrame({'name':['Peter', 'Paul', 'Mary'],
                   'food':['fish', 'beans', 'bread']},
                    columns=['name', 'food'])

In [53]:
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [54]:
df7 = pd.DataFrame({'name':['Mary', 'Joshep'],
                   'drink':['wine', 'beer']},
                    columns=['name', 'drink'])

In [55]:
df7

,name,drink
0,Mary,wine
1,Joshep,beer


In [58]:
# hacemos un join de los df, que tienen una columna en común: name
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


Por defecto, el resultado contiene la INTERSECCION de los datos df.
Es lo que se conoce como INNER JOIN

##### The how keywords

Para especificar el tipo de join, de forma explícita, utilizamos la keyword "how" cuyos valores pueden ser:

* inner: intersección de ambos 
* outer: unión de ambos, rellenando missing values con NAs
* left: join sobre el df de la izquierda
* right: join sobre el df de la derecha




In [60]:
# inner join (intersección)
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [72]:
# outer join (unión)
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joshep,NaN,beer


In [73]:
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [74]:
df7

,name,drink
0,Mary,wine
1,Joshep,beer


In [77]:
# left join (join sobre el df izquierdo)
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [76]:
# right join (join sobre el df derecho)
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joshep,NaN,beer


###  Overlapping Column Names: The suffixes keyword

Puede darse el caso que tengamos conflictos con el nombre de las columnas

In [78]:
df8 = pd.DataFrame({'name':['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank':[1,2,3,4]})

In [79]:
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [80]:
df9 = pd.DataFrame({'name':['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank':[3,4,1,2]})

In [81]:
df9

,name,rank
0,Bob,3
1,Jake,4
2,Lisa,1
3,Sue,2


In [83]:
# hacemos el join a través de la columna "name"
pd.merge(df8, df9, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,4
2,Lisa,3,1
3,Sue,4,2


Al realizar el join, tenemos un conflicto al realizar la intersección, con la columna "rank", ya que se repite en ambos df. En este caso, Pandas automáticamente cambia el nombre de las columnas, anexando un sufijo a estas para hacerlas únicas en el nuevo dataframe. En este ejemplo "rank_x" y "rank_y".

Podemos cambiar el nombre de los sufijos, utilizando la keyword: suffixes

In [84]:
# hacemos el join a través de la columna "name"
pd.merge(df8, df9, on='name', suffixes=["_L","_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,4
2,Lisa,3,1
3,Sue,4,2
